In [ ]:
# apt update && apt install -y python3-pip
!pip install transformers==4.41.2 peft==0.11.1 accelerate==0.31.0 bitsandbytes==0.43.1 gdown jupyter notebook datasets==2.20.0
!pip install decord # Thư viện để đọc video hiệu quả
!pip install triton==2.1.0
# pip install triton==3.3.1 --force-reinstall
# jupyter notebook --allow-root --port=8890


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/
# !unzip /content/drive/MyDrive/Intern_FPT/AI/DATASET/train_val_annotation.zip
# !unzip /content/drive/MyDrive/Intern_FPT/AI/DATASET/train_val_videos.zip
# !unzip /content/drive/MyDrive/Intern_FPT/AI/DATASET/test_videodatainfo.json.zip
!unzip /content/drive/MyDrive/Intern_FPT/AI/DATASET/MSRVTT/MSRVTT_Preprocessed_16frames.zip
# gdown 1k-Jk083ZEDo4T51dRr9uATdBi8Lj1-ZK


Streaming output truncated to the last 5000 lines.
  inflating: content/MSRVTT_Preprocessed_16frames/video5410/frame_0013.jpg  
  inflating: content/MSRVTT_Preprocessed_16frames/video5410/frame_0000.jpg  
   creating: content/MSRVTT_Preprocessed_16frames/video6018/
  inflating: content/MSRVTT_Preprocessed_16frames/video6018/frame_0003.jpg  
  inflating: content/MSRVTT_Preprocessed_16frames/video6018/frame_0005.jpg  
  inflating: content/MSRVTT_Preprocessed_16frames/video6018/frame_0009.jpg  
  inflating: content/MSRVTT_Preprocessed_16frames/video6018/frame_0007.jpg  
  inflating: content/MSRVTT_Preprocessed_16frames/video6018/frame_0004.jpg  
  inflating: content/MSRVTT_Preprocessed_16frames/video6018/frame_0008.jpg  
  inflating: content/MSRVTT_Preprocessed_16frames/video6018/frame_0010.jpg  
  inflating: content/MSRVTT_Preprocessed_16frames/video6018/frame_0006.jpg  
  inflating: content/MSRVTT_Preprocessed_16frames/video6018/frame_0012.jpg  
  inflating: content/MSRVTT_Preprocessed_

In [ ]:


import torch
import torch.nn as nn
from transformers import (
    CLIPVisionModelWithProjection,
    CLIPTextModelWithProjection,
    CLIPProcessor,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model
from PIL import Image
import requests
import os
import json
from tqdm import tqdm
import decord
decord.bridge.set_bridge('torch') # Đặt decord để trả về torch tensor

print("--- Cài đặt và import thư viện hoàn tất! ---")



# Tạo các thư mục cần thiết trên Drive nếu chưa có
DRIVE_PATH = "/content/drive/MyDrive/Intern_FPT/AI/DATASET/"
DATA_PATH = os.path.join(DRIVE_PATH, "/content/content")
PREPROCESSED_DATA_PATH = os.path.join(DATA_PATH, "MSRVTT_Preprocessed_16frames")
MODEL_CHECKPOINT_PATH = os.path.join(DRIVE_PATH, "training_checkpoints")

os.makedirs(PREPROCESSED_DATA_PATH, exist_ok=True)
os.makedirs(MODEL_CHECKPOINT_PATH, exist_ok=True)

print(f"Đường dẫn dự án trên Drive: {DRIVE_PATH}")
print("--- Kết nối Google Drive và tạo thư mục hoàn tất! ---")

In [ ]:


MSRVTT_TRAINVAL_VIDEO_FOLDER = os.path.join(DATA_PATH, "TrainValVideo")
MSRVTT_JSON_PATH = os.path.join(DATA_PATH, "train_val_videodatainfo.json")
NUM_FRAMES = 16

def preprocess_train_val_data():
    """
    Hàm này chỉ xử lý dữ liệu từ thư mục train-val-video và file JSON tương ứng
    để tạo ra train_data.json và val_data.json.
    """
    print("Bắt đầu tiền xử lý dữ liệu Train/Validation từ MSR-VTT...")

    if not os.path.exists(MSRVTT_TRAINVAL_VIDEO_FOLDER) or not os.path.exists(MSRVTT_JSON_PATH):
        print("LỖI: Không tìm thấy dữ liệu. Vui lòng kiểm tra lại đường dẫn:")
        print(f"Thư mục video: {MSRVTT_TRAINVAL_VIDEO_FOLDER}")
        print(f"File JSON: {MSRVTT_JSON_PATH}")
        return

    with open(MSRVTT_JSON_PATH, 'r') as f:
        data_info = json.load(f)

    sentences_list = data_info['sentences']
    videos_list = data_info['videos']
    print(f"Tìm thấy {len(videos_list)} video và {len(sentences_list)} câu chú thích trong file JSON.")

    # Tạo một từ điển để tra cứu nhanh các chú thích theo video_id
    video_to_captions = {}
    for sentence in sentences_list:
        video_id = sentence['video_id']
        caption = sentence['caption']
        if video_id not in video_to_captions:
            video_to_captions[video_id] = []
        video_to_captions[video_id].append(caption)

    train_samples = []
    val_samples = []

    # Chỉ lặp qua các video có trong file JSON (là các video train/val)
    for video_meta in tqdm(videos_list, desc="Đang xử lý các video train/val"):
        video_id = video_meta['video_id']
        split = video_meta['split']
        video_path = os.path.join(MSRVTT_TRAINVAL_VIDEO_FOLDER, f"{video_id}.mp4")

        if not os.path.exists(video_path):
            continue

        try:
            # 1. Đọc video và lấy mẫu khung hình
            vr = decord.VideoReader(video_path, ctx=decord.cpu(0))
            indices = torch.linspace(0, len(vr) - 1, NUM_FRAMES, dtype=torch.long)
            frames = vr.get_batch(indices)

            # 2. Lưu khung hình
            video_frame_folder = os.path.join(PREPROCESSED_DATA_PATH, video_id)
            os.makedirs(video_frame_folder, exist_ok=True)

            frame_paths = []
            for i, frame_tensor in enumerate(frames):
                frame_image = Image.fromarray(frame_tensor.numpy())
                # Quan trọng: Lưu đường dẫn tương đối để dễ dàng di chuyển dự án
                relative_path = os.path.join(video_id, f"frame_{i:04d}.jpg")
                full_path = os.path.join(PREPROCESSED_DATA_PATH, relative_path)
                frame_image.save(full_path)
                frame_paths.append(relative_path)

            # 3. Kết hợp và phân chia
            if video_id in video_to_captions:
                for caption in video_to_captions[video_id]:
                    sample = {
                        "video_id": video_id,
                        "frame_paths": frame_paths,
                        "caption": caption
                    }
                    if split == 'train':
                        train_samples.append(sample)
                    # Chú ý: trong MSR-VTT, split là 'validate'
                    elif split == 'validate':
                        val_samples.append(sample)

        except Exception as e:
            print(f"Lỗi khi xử lý video {video_id}: {e}")

    # 4. Lưu các file JSON
    train_file_path = os.path.join(PREPROCESSED_DATA_PATH, 'train_data.json')
    val_file_path = os.path.join(PREPROCESSED_DATA_PATH, 'val_data.json')

    with open(train_file_path, 'w') as f:
        json.dump(train_samples, f, indent=2)
    with open(val_file_path, 'w') as f:
        json.dump(val_samples, f, indent=2)

    print("\n--- Tiền xử lý Train/Validation hoàn tất! ---")
    print(f"Số mẫu huấn luyện: {len(train_samples)}")
    print(f"Số mẫu kiểm định: {len(val_samples)}")
    print(f"Dữ liệu đã được lưu tại: {PREPROCESSED_DATA_PATH}")

# ----- THỰC THI -----
# Bỏ comment và chạy một lần duy nhất.
# preprocess_train_val_data()

# Kiểm tra nhanh
train_json_path = os.path.join(PREPROCESSED_DATA_PATH, 'train_data.json')
if os.path.exists(train_json_path):
    print("Phát hiện dữ liệu train/val đã được tiền xử lý.")
    with open(train_json_path, 'r') as f:
        train_data_check = json.load(f)
        print(f"Ví dụ một mẫu dữ liệu huấn luyện:\n{json.dumps(train_data_check[0], indent=2)}")
else:
    print("Chưa có dữ liệu. Vui lòng chạy hàm preprocess_train_val_data().")

Phát hiện dữ liệu train/val đã được tiền xử lý.
Ví dụ một mẫu dữ liệu huấn luyện:
{
  "video_id": "video0",
  "frame_paths": [
    "video0/frame_0000.jpg",
    "video0/frame_0001.jpg",
    "video0/frame_0002.jpg",
    "video0/frame_0003.jpg",
    "video0/frame_0004.jpg",
    "video0/frame_0005.jpg",
    "video0/frame_0006.jpg",
    "video0/frame_0007.jpg",
    "video0/frame_0008.jpg",
    "video0/frame_0009.jpg",
    "video0/frame_0010.jpg",
    "video0/frame_0011.jpg",
    "video0/frame_0012.jpg",
    "video0/frame_0013.jpg",
    "video0/frame_0014.jpg",
    "video0/frame_0015.jpg"
  ],
  "caption": "a car is shown"
}


In [ ]:
# Tên model trên Hugging Face Hub
MODEL_NAME = "openai/clip-vit-base-patch32"
PROJECTION_DIM = 256

# --- Cấu hình lượng tử hóa 4-bit ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False, # Có thể thử bật/tắt để xem
)

# --- Định nghĩa lại class mô hình, lần này có logic lượng tử hóa bên trong ---
class DualEncoderModel(nn.Module):
    def __init__(self, model_name, projection_dim=256, bnb_config=None):
        super().__init__()

        print("Tải Vision Encoder với Quantization 4-bit...")
        self.vision_encoder = CLIPVisionModelWithProjection.from_pretrained(
            model_name,
            quantization_config=bnb_config
        )

        print("Tải Text Encoder với Quantization 4-bit...")
        self.text_encoder = CLIPTextModelWithProjection.from_pretrained(
            model_name,
            quantization_config=bnb_config
        )

        vision_hidden_size = self.vision_encoder.config.projection_dim
        text_hidden_size = self.text_encoder.config.projection_dim

        self.video_projection = nn.Linear(vision_hidden_size, projection_dim, bias=False)
        self.text_projection = nn.Linear(text_hidden_size, projection_dim, bias=False)

    def forward(self, pixel_values=None, input_ids=None, attention_mask=None, **kwargs):
        # Logic forward giữ nguyên như trước
        video_embeds = None
        if pixel_values is not None:
            batch_size, num_frames, C, H, W = pixel_values.shape
            pixel_values = pixel_values.view(-1, C, H, W)
            vision_outputs = self.vision_encoder(pixel_values=pixel_values)
            frame_embeds = vision_outputs.image_embeds
            video_embeds = frame_embeds.view(batch_size, num_frames, -1).mean(dim=1)
            video_embeds = self.video_projection(video_embeds)
            video_embeds = video_embeds / video_embeds.norm(dim=-1, keepdim=True)

        text_embeds = None
        if input_ids is not None:
            text_outputs = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
            text_embeds = text_outputs.text_embeds
            text_embeds = self.text_projection(text_embeds)
            text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)

        return video_embeds, text_embeds

# --- Khởi tạo mô hình đã được lượng tử hóa ---
print("Khởi tạo mô hình đã lượng tử hóa...")
quantized_model = DualEncoderModel(
    model_name=MODEL_NAME,
    projection_dim=PROJECTION_DIM,
    bnb_config=bnb_config
)

# --- Áp dụng LoRA lên mô hình đã lượng tử hóa ---
print("\nÁp dụng LoRA...")
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "text_projection", "video_projection"],
    lora_dropout=0.05,
    bias="none",
    task_type="FEATURE_EXTRACTION"
)

# Sử dụng prepare_model_for_kbit_training để đảm bảo khả năng tương thích
from peft import prepare_model_for_kbit_training
# Bước này chuẩn bị mô hình để gradient checkpointing và đảm bảo các lớp layer norm ổn định
model_for_lora = prepare_model_for_kbit_training(quantized_model)

lora_model = get_peft_model(model_for_lora, lora_config)

print("\n--- Tổng quan các tham số có thể huấn luyện sau khi áp dụng QLoRA ---")
lora_model.print_trainable_parameters()
print("\n--- Xây dựng lại mô hình hoàn tất! Sẵn sàng để huấn luyện. ---")

Khởi tạo mô hình đã lượng tử hóa...
Tải Vision Encoder với Quantization 4-bit...


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Tải Text Encoder với Quantization 4-bit...

Áp dụng LoRA...

--- Tổng quan các tham số có thể huấn luyện sau khi áp dụng QLoRA ---
trainable params: 1,515,520 || all params: 153,054,976 || trainable%: 0.9902

--- Xây dựng lại mô hình hoàn tất! Sẵn sàng để huấn luyện. ---


In [ ]:

from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import os
from PIL import Image

# Tải CLIP Processor để xử lý ảnh và text theo đúng cách của CLIP
# Processor sẽ bao gồm cả ImageProcessor và Tokenizer
processor = CLIPProcessor.from_pretrained(MODEL_NAME)

class MSRVTT_Dataset(Dataset):
    def __init__(self, json_file_path, preprocessed_data_folder, processor):
        """
        Args:
            json_file_path (str): Đường dẫn đến file train_data.json hoặc val_data.json
            preprocessed_data_folder (str): Đường dẫn đến thư mục chứa các khung hình đã xử lý
            processor: CLIPProcessor để xử lý ảnh và text
        """
        print(f"Tải dữ liệu từ: {json_file_path}")
        with open(json_file_path, 'r') as f:
            self.data = json.load(f)
        self.preprocessed_data_folder = preprocessed_data_folder
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        # --- Xử lý Video (đọc các khung hình) ---
        frame_paths = sample['frame_paths']
        # Lấy đường dẫn tuyệt đối
        absolute_frame_paths = [os.path.join(self.preprocessed_data_folder, p) for p in frame_paths]

        # Đọc các ảnh và đưa vào danh sách
        video_frames = [Image.open(p).convert("RGB") for p in absolute_frame_paths]

        # Xử lý video frames bằng processor
        # Processor sẽ resize, normalize và chuyển thành tensor
        # Do processor.vision_processor chỉ xử lý một list ảnh, nó sẽ tự động stack chúng lại
        pixel_values = self.processor(
            images=video_frames,
            return_tensors="pt"
        ).pixel_values

        # --- Xử lý Text ---
        caption = sample['caption']
        text_inputs = self.processor(
            text=caption,
            padding="max_length", # hoặc 'longest'
            truncation=True,
            max_length=77, # Chiều dài max của CLIP text encoder
            return_tensors="pt"
        )

        return {
            "pixel_values": pixel_values,
            "input_ids": text_inputs.input_ids.squeeze(0), # Bỏ chiều batch thừa
            "attention_mask": text_inputs.attention_mask.squeeze(0)
        }

# Khởi tạo dataset
train_json_path = os.path.join(PREPROCESSED_DATA_PATH, 'train_data.json')
train_dataset = MSRVTT_Dataset(
    json_file_path=train_json_path,
    preprocessed_data_folder=PREPROCESSED_DATA_PATH,
    processor=processor
)

# Tạo DataLoader
BATCH_SIZE = 32 # Bắt đầu với 32. Nếu VRAM còn nhiều, có thể tăng lên 48 hoặc 64.
# Batch size nhỏ vì VRAM hạn chế
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f"\n--- Đã tạo Dataset và DataLoader cho tập train. Số mẫu: {len(train_dataset)} ---")
# Kiểm tra nhanh một batch
sample_batch = next(iter(train_dataloader))
print("Kích thước pixel_values trong 1 batch:", sample_batch["pixel_values"].shape)
print("Kích thước input_ids trong 1 batch:", sample_batch["input_ids"].shape)


# @title 4.2 Định nghĩa Loss Function (InfoNCE)
class InfoNCELoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        # Khởi tạo nhiệt độ như một tham số có thể học
        self.temperature = nn.Parameter(torch.tensor(temperature))

    def forward(self, video_embeds, text_embeds):
        # video_embeds: (batch_size, embed_dim)
        # text_embeds: (batch_size, embed_dim)

        # Tính ma trận tương đồng (cosine similarity)
        # embedding đã được L2-normalized trong mô hình
        logits = torch.matmul(video_embeds, text_embeds.T) / self.temperature

        # Ground truth: các cặp dương tính nằm trên đường chéo chính
        batch_size = video_embeds.shape[0]
        labels = torch.arange(batch_size, device=video_embeds.device)

        # Tính cross-entropy loss cho cả hai chiều
        loss_v2t = nn.functional.cross_entropy(logits, labels)
        loss_t2v = nn.functional.cross_entropy(logits.T, labels)

        # Loss cuối cùng là trung bình của hai chiều
        loss = (loss_v2t + loss_t2v) / 2.0
        return loss

loss_fn = InfoNCELoss()
print("\n--- Định nghĩa InfoNCE Loss hoàn tất! ---")

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Tải dữ liệu từ: /content/content/MSRVTT_Preprocessed_16frames/train_data.json

--- Đã tạo Dataset và DataLoader cho tập train. Số mẫu: 130260 ---
Kích thước pixel_values trong 1 batch: torch.Size([2, 16, 3, 224, 224])
Kích thước input_ids trong 1 batch: torch.Size([2, 77])

--- Định nghĩa InfoNCE Loss hoàn tất! ---


In [ ]:
#==============================================================================
# MỤC 3 (TIẾP THEO): CẤU HÌNH VÀ CHUẨN BỊ HUẤN LUYỆN (CHO RTX 4090)
#==============================================================================
# @title 3.3 Chuẩn bị Dataloader, Optimizer, Scheduler (Tối ưu cho 4090)

from accelerate import Accelerator
from transformers import get_linear_schedule_with_warmup
import bitsandbytes as bnb

# ---------------------------------------------------
# 1. CẤU HÌNH CÁC SIÊU THAM SỐ
# ---------------------------------------------------
NUM_EPOCHS = 3
LEARNING_RATE = 5e-5 # Có thể thử một LR nhỏ hơn một chút vì batch size lớn hơn
GRADIENT_ACCUMULATION_STEPS = 1 # Cập nhật sau mỗi batch để có tốc độ nhanh nhất!

# Đường dẫn lưu checkpoint
CHECKPOINT_FOLDER = os.path.join(DRIVE_PATH, "checkpoints_retrieval_4090")
os.makedirs(CHECKPOINT_FOLDER, exist_ok=True)

print(f"Cấu hình cho RTX 4090:")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Gradient Accumulation Steps: {GRADIENT_ACCUMULATION_STEPS}")
print(f"  Effective Batch Size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")

# ---------------------------------------------------
# 2. CHUẨN BỊ DATALOADER
# ---------------------------------------------------
processor = CLIPProcessor.from_pretrained(MODEL_NAME)
train_dataset = MSRVTT_Dataset(
    json_file_path=os.path.join(PREPROCESSED_DATA_PATH, 'train_data.json'),
    preprocessed_data_folder=PREPROCESSED_DATA_PATH,
    processor=processor
)
# num_workers > 0 sẽ tăng tốc độ nạp dữ liệu bằng cách dùng nhiều tiến trình
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# ---------------------------------------------------
# 3. CHUẨN BỊ OPTIMIZER VÀ SCHEDULER
# ---------------------------------------------------
# Vẫn sử dụng PagedAdamW8bit để hiệu quả về bộ nhớ
optimizer = bnb.optim.PagedAdamW8bit(
    lora_model.parameters(), # lora_model đã được tạo ở bước trước
    lr=LEARNING_RATE
)

# Tính toán tổng số bước huấn luyện
num_update_steps_per_epoch = len(train_dataloader) // GRADIENT_ACCUMULATION_STEPS
total_training_steps = NUM_EPOCHS * num_update_steps_per_epoch

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0, # Bắt đầu với 0 warmup steps cho đơn giản
    num_training_steps=total_training_steps,
)

# ---------------------------------------------------
# 4. KHỞI TẠO ACCELERATOR VÀ CHUẨN BỊ
# ---------------------------------------------------
accelerator = Accelerator(
    mixed_precision="bf16",  # <-- SỬ DỤNG BFLOAT16 CHO 4090
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
)

lora_model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    lora_model, optimizer, train_dataloader, lr_scheduler
)

print("\n--- Thiết lập Accelerator, Optimizer, Scheduler hoàn tất! Sẵn sàng huấn luyện với cấu hình mới. ---")

--- Thiết lập Accelerator, Optimizer, Scheduler hoàn tất! Sẵn sàng huấn luyện. ---


/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:477: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:

lora_model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    lora_model, optimizer, train_dataloader, lr_scheduler
)

# ----- LOGIC TẢI LẠI TRẠNG THÁI (ĐÃ SỬA) -----
# Tìm checkpoint mới nhất
# Sử dụng os.path.join để đảm bảo tương thích
checkpoint_search_path = os.path.join(CHECKPOINT_FOLDER, "checkpoint_step_*")
checkpoints = sorted(glob.glob(checkpoint_search_path))

# --- KHỞI TẠO BIẾN Ở NGOÀI KHỐI IF ---
start_epoch = 0
start_step = 0
resume_step_in_epoch = 0

if checkpoints:
    latest_checkpoint = checkpoints[-1]
    print(f"Phát hiện checkpoint. Tải lại trạng thái từ: {latest_checkpoint}")
    # accelerator.load_state() cần được gọi trước khi tiếp tục
    # Đảm bảo mô hình đang ở trên đúng device
    accelerator.load_state(latest_checkpoint)

    start_step = int(latest_checkpoint.split("_")[-1])
    # Tính toán lại epoch bắt đầu và vị trí trong dataloader
    # Chia cho len(train_dataloader) đã được `prepare`
    steps_per_epoch = len(train_dataloader)
    start_epoch = start_step // steps_per_epoch
    resume_step_in_epoch = start_step % steps_per_epoch
    print(f"Tiếp tục từ Epoch {start_epoch + 1}, Bước {resume_step_in_epoch}")
else:
    print("Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu.")
    # Các giá trị mặc định đã được khởi tạo ở trên

total_training_steps = len(train_dataloader) * NUM_EPOCHS
completed_steps = start_step

print("Bắt đầu quá trình fine-tuning bằng QLoRA...")
lora_model.train()

for epoch in range(start_epoch, NUM_EPOCHS):
    # Nếu tiếp tục từ giữa epoch, ta cần bỏ qua các batch đã xử lý
    # Chỉ bỏ qua ở epoch đầu tiên tiếp tục
    if epoch == start_epoch and resume_step_in_epoch > 0:
        print(f"Bỏ qua {resume_step_in_epoch} batch đầu tiên của epoch {epoch + 1}...")
        active_dataloader = accelerator.skip_first_batches(train_dataloader, resume_step_in_epoch)
    else:
        active_dataloader = train_dataloader
        # Đặt lại cho các epoch sau
        resume_step_in_epoch = 0


    progress_bar = tqdm(
        active_dataloader,
        desc=f"Epoch {epoch + 1}/{NUM_EPOCHS}",
        initial=resume_step_in_epoch, # Chỉ đặt initial cho epoch đầu tiên
        total=len(train_dataloader)
    )

    for step, batch in enumerate(progress_bar):
        with accelerator.accumulate(lora_model):
            # Forward pass
            video_embeds, text_embeds = lora_model(**batch)
            loss = loss_fn(video_embeds, text_embeds)

            # Backward pass
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            # Cập nhật thanh tiến trình
            progress_bar.set_postfix(loss=loss.item())

        # Logic lưu checkpoint dựa trên tổng số bước đã hoàn thành
        # accelerator.sync_gradients phải được gọi nếu dùng FSDP/DeepSpeed
        # nhưng với accumulate thì không cần.
        current_step_in_epoch = resume_step_in_epoch + step + 1
        global_step = epoch * len(train_dataloader) + current_step_in_epoch

        # Lưu lại sau mỗi N bước tối ưu
        if global_step % 2000 == 0 and accelerator.is_main_process:
            output_dir = os.path.join(CHECKPOINT_FOLDER, f"checkpoint_step_{global_step}")
            print(f"\nĐã hoàn thành {global_step} bước. Đang lưu trạng thái tại: {output_dir}")
            accelerator.save_state(output_dir)

    print(f"Hoàn thành Epoch {epoch + 1}")

print("\n--- HUẤN LUYỆN HOÀN TẤT! ---")
if accelerator.is_main_process:
    final_checkpoint_dir = os.path.join(CHECKPOINT_FOLDER, "final_checkpoint")
    accelerator.save_state(final_checkpoint_dir)
    print(f"Đã lưu trạng thái huấn luyện cuối cùng tại: {final_checkpoint_dir}")

Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu.
Bắt đầu quá trình fine-tuning bằng QLoRA...


Epoch 1/5:   1%|          | 499/65130 [03:28<6:35:58,  2.72it/s, loss=0.609]


Đã hoàn thành 500 bước. Đang lưu trạng thái tại: /content/drive/MyDrive/Intern_FPT/AI/DATASET/training_checkpoints/checkpoint_step_500


Epoch 1/5:   2%|▏         | 999/65130 [07:04<10:12:43,  1.74it/s, loss=0.65] 


Đã hoàn thành 1000 bước. Đang lưu trạng thái tại: /content/drive/MyDrive/Intern_FPT/AI/DATASET/training_checkpoints/checkpoint_step_1000


...

Epoch 1/5:  40%|███▉      | 25999/65130 [2:58:01<3:53:20,  2.80it/s, loss=0.353]


Đã hoàn thành 26000 bước. Đang lưu trạng thái tại: /content/drive/MyDrive/Intern_FPT/AI/DATASET/training_checkpoints/checkpoint_step_26000


Epoch 1/5:  41%|████      | 26499/65130 [3:01:25<4:39:24,  2.30it/s, loss=0.103] 


Đã hoàn thành 26500 bước. Đang lưu trạng thái tại: /content/drive/MyDrive/Intern_FPT/AI/DATASET/training_checkpoints/checkpoint_step_26500


Epoch 1/5:  41%|████      | 26596/65130 [3:02:09<3:56:16,  2.72it/s, loss=0.122]

In [ ]:

from peft import PeftModel

@torch.no_grad()
def evaluate_retrieval(model, test_dataloader, accelerator):
    model.eval()
    all_video_embeds = []
    all_text_embeds = []

    print("Tạo embedding cho toàn bộ tập test...")
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        # Chỉ cần tính video_embeds và text_embeds riêng biệt
        video_embeds, _ = model(pixel_values=batch["pixel_values"])
        _, text_embeds = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])

        # Dùng accelerator.gather để tổng hợp kết quả từ tất cả các tiến trình (nếu có)
        video_embeds = accelerator.gather(video_embeds)
        text_embeds = accelerator.gather(text_embeds)

        all_video_embeds.append(video_embeds.cpu())
        all_text_embeds.append(text_embeds.cpu())

    all_video_embeds = torch.cat(all_video_embeds, dim=0)
    all_text_embeds = torch.cat(all_text_embeds, dim=0)

    # Tính toán similarity
    sim_matrix = torch.matmul(all_text_embeds, all_video_embeds.T)

    # Ground truth: mỗi text i phải khớp với video i
    # Lưu ý: Cần đảm bảo dataloader không shuffle và các cặp (text, video) là duy nhất
    # Với MSR-VTT, mỗi video có nhiều caption, nên cần xử lý phức tạp hơn
    # Ở đây ta làm một phiên bản đơn giản hóa, giả định mỗi text có 1 video đúng
    # Một cách tiếp cận đúng hơn là phải nhóm các caption theo video_id

    print("Tính toán các chỉ số R@K...")
    # Text-to-Video Retrieval
    ranks = []
    for i in range(sim_matrix.shape[0]):
      # Sắp xếp các video theo độ tương đồng với text i
      sorted_indices = torch.argsort(sim_matrix[i, :], descending=True)
      # Tìm rank của video đúng (video i)
      rank = (sorted_indices == i).nonzero(as_tuple=True)[0].item()
      ranks.append(rank + 1)

    ranks = torch.tensor(ranks)
    r1 = (ranks <= 1).float().mean().item() * 100
    r5 = (ranks <= 5).float().mean().item() * 100
    r10 = (ranks <= 10).float().mean().item() * 100
    mdr = torch.median(ranks).item()

    return {"R@1": r1, "R@5": r5, "R@10": r10, "MdR": mdr}

# Tải mô hình đã fine-tune
MODEL_CHECKPOINT_PATH = ''
# Chọn checkpoint tốt nhất (ví dụ: epoch cuối cùng)
best_adapter_path = os.path.join(MODEL_CHECKPOINT_PATH, "retrieval_adapter_epoch_3")

if os.path.exists(best_adapter_path):
    print(f"Tải adapter đã huấn luyện từ: {best_adapter_path}")
    base_model = DualEncoderModel(model_name=MODEL_NAME, projection_dim=PROJECTION_DIM)
    eval_model = PeftModel.from_pretrained(base_model, best_adapter_path)
    eval_model = eval_model.merge_and_unload() # Hợp nhất để suy luận nhanh hơn
    eval_model.eval()

    # Chuẩn bị dữ liệu test (cần tạo test_data.json tương tự như train/val)
    # Giả sử bạn đã có `test_data.json`
    test_json_path = os.path.join(PREPROCESSED_DATA_PATH, 'test_data.json') # Cần tạo file này
    if os.path.exists(test_json_path):
      test_dataset = MSRVTT_Dataset(
          json_file_path=test_json_path,
          preprocessed_data_folder=PREPROCESSED_DATA_PATH,
          processor=processor
      )
      # Dùng batch size lớn hơn khi đánh giá
      test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

      eval_model, test_dataloader = accelerator.prepare(eval_model, test_dataloader)

      results = evaluate_retrieval(eval_model, test_dataloader, accelerator)
      print("\n--- KẾT QUẢ ĐÁNH GIÁ ---")
      print(json.dumps(results, indent=2))
    else:
      print("Không tìm thấy file test_data.json. Vui lòng tạo file này để đánh giá.")
else:
    print(f"Không tìm thấy checkpoint tại {best_adapter_path}. Vui lòng huấn luyện trước.")